In [9]:
import pandas as pd
import json
import requests
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

%config Completer.use_jedi = False
pd.options.display.max_columns = 40
pd.options.display.max_rows = 50

## Lemmatization

Load the lemmatized ESCO skill descriptions

In [10]:
df = pd.read_json('lemmatized_esco_data.json')
df.head()

,preferredLabel,preferredLabelLemmatized,preferredLabelPOS,alternativeLabel,alternativeLabelLemmatized,alternativeLabelPOS,description,descriptionLemmatized,descriptionPOS
0,analysoida röntgenkuvia,"[analysoida, röntgen#kuva]","[VERB, NOUN]",[],[],[],Analysoida röntgenkuvia potilaiden ongelmien s...,"[analysoida, röntgen#kuva, potilas, ongelma...","[VERB, NOUN, NOUN, NOUN, NOUN, CCONJ, NO..."
1,myydä kelloja,"[myydä, kello]","[VERB, NOUN]",[],[],[],"Myydä kelloja, rannekelloja tai niihin liittyv...","[myydä, kello, , , ranne#kello, tai, se, ...","[VERB, NOUN, PUNCT, NOUN, CCONJ, PRON, V..."
2,valvoa myynninjälkeisiä tilastoja,"[valvoa, myynnin#jälkeinen, tilasto]","[VERB, ADJ, NOUN]",[seurata palautetilastoja],"[[seurata, palaute#tilasto]]","[[VERB, NOUN]]",Pitää silmällä myynninjälkeistä palautetta ja ...,"[pitää, silmä, myynnin#jälkeinen, palaute, ...","[VERB, NOUN, ADJ, NOUN, CCONJ, VERB, NOU..."
3,käyttää lakantuotantolaitteita,"[käyttää, lakan#tuotanto#laite]","[VERB, NOUN]",[],[],[],Lakan valmistuksessa käytettäviä raaka-aineita...,"[lakka, valmistus, käyttää, raaka#aine, su...","[NOUN, NOUN, VERB, NOUN, VERB, PUNCT, VE..."
4,hoitaa kuivaustunneleita,"[hoitaa, kuivaus#tunneli]","[VERB, NOUN]",[],[],[],"Hoitaa tunneleita, joissa kuivataan savituotte...","[hoitaa, tunnel, , , joka, kuivata, savi#...","[VERB, NOUN, PUNCT, PRON, VERB, NOUN, PU..."


In [11]:
# For this lemmatization to work you need to start the TurkuNLP docker container in server mode by running
# docker run -d -p 15000:7689 turkunlp/turku-neural-parser:latest-fi-en-sv-cpu server fi_tdt parse_plaintext

def lemmatize(text):
    response = requests.post('http://localhost:15000', data=text.encode('utf-8'), headers={'Content-Type': 'text/plain; charset=utf-8'})
    
    lemmatized = []
    POS = []
    
    for line in response.text.split('\n'):
        if not line:
            continue
        if line[0] == '#':
            continue
        
        l = line.split('\t')
        lemmatized.append(l[2])
        POS.append(l[3])

    return lemmatized, POS

In [17]:
data = df[['preferredLabel', 'preferredLabelLemmatized', 'preferredLabelPOS', 'alternativeLabel', 'alternativeLabelLemmatized', 'alternativeLabelPOS', 'description', 'descriptionLemmatized', 'descriptionPOS']]
data.head()

,preferredLabel,preferredLabelLemmatized,preferredLabelPOS,alternativeLabel,alternativeLabelLemmatized,alternativeLabelPOS,description,descriptionLemmatized,descriptionPOS
0,analysoida röntgenkuvia,"[analysoida, röntgen#kuva]","[VERB, NOUN]",[],[],[],Analysoida röntgenkuvia potilaiden ongelmien s...,"[analysoida, röntgen#kuva, potilas, ongelma...","[VERB, NOUN, NOUN, NOUN, NOUN, CCONJ, NO..."
1,myydä kelloja,"[myydä, kello]","[VERB, NOUN]",[],[],[],"Myydä kelloja, rannekelloja tai niihin liittyv...","[myydä, kello, , , ranne#kello, tai, se, ...","[VERB, NOUN, PUNCT, NOUN, CCONJ, PRON, V..."
2,valvoa myynninjälkeisiä tilastoja,"[valvoa, myynnin#jälkeinen, tilasto]","[VERB, ADJ, NOUN]",[seurata palautetilastoja],"[[seurata, palaute#tilasto]]","[[VERB, NOUN]]",Pitää silmällä myynninjälkeistä palautetta ja ...,"[pitää, silmä, myynnin#jälkeinen, palaute, ...","[VERB, NOUN, ADJ, NOUN, CCONJ, VERB, NOU..."
3,käyttää lakantuotantolaitteita,"[käyttää, lakan#tuotanto#laite]","[VERB, NOUN]",[],[],[],Lakan valmistuksessa käytettäviä raaka-aineita...,"[lakka, valmistus, käyttää, raaka#aine, su...","[NOUN, NOUN, VERB, NOUN, VERB, PUNCT, VE..."
4,hoitaa kuivaustunneleita,"[hoitaa, kuivaus#tunneli]","[VERB, NOUN]",[],[],[],"Hoitaa tunneleita, joissa kuivataan savituotte...","[hoitaa, tunnel, , , joka, kuivata, savi#...","[VERB, NOUN, PUNCT, PRON, VERB, NOUN, PU..."


In [22]:
def get_documents(data):
    documents = {}
    for i in data.index:
        document_words = []
        
        preferred_label = data['preferredLabel'][i]
        preferred_label_lemmatized = data['preferredLabelLemmatized'][i]
        preferred_label_POS = data['preferredLabelPOS'][i]
        for j in range(len(preferred_label_lemmatized)):
            if preferred_label_POS[j].strip() in ['VERB', 'NOUN']:
                document_words.append(preferred_label_lemmatized[j].strip())
                
        alternative_label = data['alternativeLabel'][i]
        alternative_label_lemmatized = data['alternativeLabelLemmatized'][i]
        alternative_label_POS = data['alternativeLabelPOS'][i]
        for j in range(len(alternative_label_lemmatized)):
            for k in range(len(alternative_label_lemmatized[j])):
                if alternative_label_POS[j][k].strip() in ['VERB', 'NOUN']:
                    document_words.append(alternative_label_lemmatized[j][k].strip())
                
        description_lemmatized = [item for item in data['descriptionLemmatized'][i] if item != ' ']
        description_POS = data['descriptionPOS'][i]
        for j in range(len(description_lemmatized)):
            if description_POS[j].strip() in ['VERB', 'NOUN']:
                document_words.append(description_lemmatized[j].strip())
        
        document = ' '.join(document_words)
        
        documents[preferred_label] = document
        
    return documents

documents = get_documents(data)

In [23]:
documents

{'analysoida röntgenkuvia': 'analysoida röntgen#kuva analysoida röntgen#kuva potilas ongelma selvittäminen tulos tulkitseminen',
 'myydä kelloja': 'myydä kello myydä kello ranne#kello liittyä tarvike asiakas mieltymys',
 'valvoa myynninjälkeisiä tilastoja': 'valvoa tilasto seurata palaute#tilasto pitää silmä palaute seurata asiakas#tyytyväisyys valitus kirjata puhelu tieto analysointi',
 'käyttää lakantuotantolaitteita': 'käyttää lakan#tuotanto#laite lakka valmistus käyttää raaka#aine sulata keittävä sekoittaa kone käyttäminen sekoittaa aines#osa kumi teollisuus#bensiini öljy',
 'hoitaa kuivaustunneleita': 'hoitaa kuivaus#tunneli hoitaa tunnel kuivata savi#tuote tiili katto#tiili jatko#jalostus uuni',
 'tutkia ilmastointijärjestelmää': 'tutkia ilmastointi#järjestelmä tarkastaa ilma#vaihto#järjestelmä toiminnallisuus palo#turvallisuus osa',
 'huoltaa vedenkäsittelylaitteita': 'huoltaa veden#käsittely#laite tehdä vesi jäte#vesi puhdistus käsittely käyttää laite huolto#työ huolto',
 'purk

In [30]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(list(documents.values()))

In [31]:
# Here you only want to use vectorizer.transform and not fit_transform since the tfidf vectorizer has already been fitted with all the descriptions
def cosine_similarity(text1, text2):
    tmp = vectorizer.transform([text1, text2])
    return ((tmp * tmp.T).A)[0, 1]

In [32]:
def get_similarities(text):
    text_lemmatized, _ = lemmatize(text)
    text_preprocessed = ' '.join(text_lemmatized)
    
    similarities = {}
    
    for key, value in documents.items():
        similarities[key] = cosine_similarity(text_preprocessed, value)

    return similarities    

In [33]:
d = get_similarities('hyviä vuorovaikutustaitoja')
d_sorted = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
d_sorted

{'osoittaa sosiaalisia taitoja': 0.6608136840197328,
 'molekyylibiologia': 0.6316587700513385,
 'ohjelmiston vuorovaikutuksen suunnittelu': 0.560586210675245,
 'opettaa yrityksessä tarvittavia työskentelytaitoja': 0.5445002795457043,
 'ihmisen ja tietokoneen välinen vuorovaikutus': 0.5309527189603742,
 'aikatauluttaa vuorot': 0.4457319714703615,
 'tukea sosiaalipalveluiden käyttäjiä taitojen hallinnassa': 0.43563269312051395,
 'hallita kaivojen vuorovaikutusprosessit': 0.4341653421692856,
 'olla huomaavainen': 0.4142654521289544,
 'lääkkeiden vaikutuksen hallinta': 0.409578079760355,
 'koordinoida vuoroja': 0.40503967335243735,
 'suunnitella käyttöliittymää': 0.3983804807191749,
 'käyttää kliinistä kiropraktista ammattitaitoa urheiluvammojen hoitamiseen': 0.3947580856896732,
 'olla vuorovaikutuksessa kohdeyhteisön kanssa': 0.37789049390818774,
 'työskennellä vuoroissa': 0.36406654984586645,
 'järjestää harjoituksia motoristen taitojen kehittämiseksi': 0.3607894324947602,
 'hallita toim

## Stemming

In [6]:
import re
import nltk

stemmer = nltk.stem.snowball.SnowballStemmer('finnish')
stopwords = nltk.corpus.stopwords.words('finnish')

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def tokenize(text):
    return re.split(' |,|\!|\(|\)|\/|-|:|\r\n', data)


def has_numbers(string):
    return any(char.isdigit() for char in string)


def has_at_sign(string):
    return '@' in string


def clean_tokens(tokens):
    tokens = [token for token in tokens if token and not any([has_numbers(token), has_at_sign(token)])]
    
    return tokens


def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]


def tokenizer(text):
    tokens = re.split(' |,|\!|\(|\)|\/|-|:|\r\n', text.lower())
    tokens = [token for token in tokens if token and not any([has_numbers(token), has_at_sign(token)])]
    tokens = remove_stopwords(tokens)
    tokens = [token.replace('.', '') for token in tokens]
    tokens = stem_tokens(tokens)
    
    return tokens

In [11]:
def get_stemmed_documents(data):
    documents = {}
    for i in data.index:
        document_words = []
        
        preferred_label = data['preferredLabel'][i]
        document_words.extend(tokenizer(preferred_label))

        description = data['description'][i]
        document_words.extend(tokenizer(description))
        
        document = ' '.join(document_words)
        documents[preferred_label] = document
        
    return documents

stemmed_documents = get_stemmed_documents(data)

In [12]:
stemmed_vectorizer = TfidfVectorizer()
stemmed_tfidf = stemmed_vectorizer.fit_transform(list(stemmed_documents.values()))

In [13]:
def stemmed_cosine_similarity(text1, text2):
    tmp = stemmed_vectorizer.transform([text1, text2])
    return ((tmp * tmp.T).A)[0, 1]

def get_stemmed_similarities(text):
    text_tokens = tokenizer(text)
    text_stemmed = ' '.join(text_tokens)
    
    similarities = {}
    
    for key, value in stemmed_documents.items():
        similarities[key] = stemmed_cosine_similarity(text_stemmed, value)
    
    return similarities    

In [14]:
d = get_stemmed_similarities('hyviä vuorovaikutustaitoja')
d_sorted = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
d_sorted

{'jakaa hyviä käytäntöjä tytäryhtiöihin': 0.40257129562284893,
 'toimia hyvien tapojen mukaisesti pelaajien kanssa': 0.3978977424934986,
 'testata sähkönsiirron käytäntöjä': 0.3571446219340207,
 'edistää hyviä tapoja viestintähäiriöiden ehkäisemiseksi': 0.342163021971418,
 'valmistaa lähetykset hyvissä ajoin': 0.31356493005875136,
 'ostaa osia': 0.2780904871313674,
 'ylläpitää tietokannan turvallisuutta': 0.2659599076128245,
 'tehdä myyntianalyysi': 0.26555130741278515,
 'kirjoittaa tuotantoraportteja': 0.2627778312914531,
 'tyhjiötislausprosessit': 0.2546292758114418,
 'joukko-oppi': 0.2530449602924419,
 'käyttää myyntiennusteohjelmistoja': 0.2513880699316718,
 'ylläpitää ihmissuhteita työpaikalla': 0.2465451896850979,
 'valvoa vieraiden pesulapalvelua': 0.24551351990442863,
 'suhtautua potilaisiin hyvin': 0.24138000486814837,
 'noudattaa hyvää valmistustapaa': 0.24126332646379683,
 'kehittää projektisuunnitelma': 0.24088578516168052,
 'toimia ensihoitotilanteessa': 0.2308008520717550

## TS-SS (triange's area similarity and sector's area similarity combination)

Experiment with a different similarity metric.

https://github.com/taki0112/Vector_Similarity

Note the current implementation runs a long time.

In [74]:
import math
from IPython.display import display, clear_output

def vector_size(vec):
    return math.sqrt(sum(math.pow(v, 2) for v in vec))

def inner_product(vec1, vec2):
    return sum(v1 * v2 for v1, v2 in zip(vec1, vec2))

def cosine(vec1, vec2):
    return inner_product(vec1, vec2) / (vector_size(vec1) * vector_size(vec2))

def theta(vec1, vec2):
    return math.acos(cosine(vec1, vec2)) + math.radians(10)

def triangle(vec1, vec2):
    angle_rad = math.radians(theta(vec1, vec2))
    return (vector_size(vec1) * vector_size(vec2) * math.sin(angle_rad)) / 1

def magnitude_difference(vec1, vec2):
    return abs(vector_size(vec1) - vector_size(vec2))

def euclidean(vec1, vec2):
    return math.sqrt(sum(math.pow((v1 - v2), 2) for v1, v2 in zip(vec1, vec2)))

def sector(vec1, vec2):
    ed = euclidean(vec1, vec2)
    md = magnitude_difference(vec1, vec2)
    angle = theta(vec1, vec2)
    return math.pi * math.pow((ed + md), 2) * angle / 360

def TS_SS(vec1, vec2):
    return triangle(vec1, vec2) * sector(vec1, vec2)

def TS_SS_similarity(text1, text2):
    sparse = stemmed_vectorizer.transform([text1, text2])
    
    vec1 = sparse[0].todense().A[0]
    vec2 = sparse[1].todense().A[0]
    
    return TS_SS(vec1, vec2)

def get_TS_SS_similarities(text):
    text_tokens = tokenizer(text)
    text_stemmed = ' '.join(text_tokens)
    
    similarities = {}
    
    i = 1
    for key, value in stemmed_documents.items():
        clear_output(wait=True)
        print(f"{i}/{len(stemmed_documents)}")
        similarities[key] = TS_SS_similarity(text_stemmed, value)
        i += 1

    return similarities

text = 'jakaa postia'

d = get_TS_SS_similarities(text)
d_sorted = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
d_sorted

12566/12566


{'selviytyä merellä tilanteessa, jossa laiva joudutaan jättämään': 0.0009277742258158227,
 'suunnitella tapahtumamarkkinointia markkinointikampanjoihin': 0.0009277742258158226,
 'työskennellä psykologisten käytösmallien parissa': 0.0009277742258158226,
 'rahtiteollisuus': 0.0009277742258158226,
 'noudattaa työaikataulua': 0.0009277742258158219,
 'antaa selonteko oikeuden virkailijoille': 0.0009277742258158219,
 'kehittää teknisten tekstiilien teknisiä tietoja': 0.0009277742258158219,
 'kestää ulosteita': 0.0009277742258158219,
 'mitata alusten vetoisuus': 0.0009277742258158219,
 'huolehtia kuvauslaitteista': 0.0009277742258158219,
 'toimia kuiskaajana esityksissä': 0.0009277742258158219,
 'antaa nitraattien aiheuttamaa pilaantumista koskevia neuvoja': 0.0009277742258158219,
 'asentaa laskettu sisäkatto': 0.0009277742258158219,
 'mausteiden valmistusprosessit': 0.0009277742258158219,
 'markkinaosapuolet': 0.0009277742258158219,
 'aluksen turvavarusteet': 0.0009277742258158219,
 'osoitta